## First Document (ml.pdf)

In [1]:
from langchain_community.document_loaders import PDFMinerLoader


pdf_loader = PDFMinerLoader("./data/ml.pdf")
documents = pdf_loader.load()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
chunked_documents = text_splitter.split_documents(documents)

len(chunked_documents)

2460

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS



embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [4]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from uuid import uuid4

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

uuids = [str(uuid4()) for _ in range(len(chunked_documents))]
vector_store.add_documents(documents=chunked_documents, ids=uuids)
vector_store.save_local("faiss_index1")

## Second Document (ml2.pdf)

In [5]:
from langchain_community.document_loaders import PDFMinerLoader


pdf_loader2 = PDFMinerLoader("./data/xai.pdf")
documents2 = pdf_loader2.load()

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter2 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
chunked_documents2 = text_splitter2.split_documents(documents2)

len(chunked_documents2)

704

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS



embeddings2 = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [8]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from uuid import uuid4

index2 = faiss.IndexFlatL2(len(embeddings2.embed_query("hello world")))

vector_store2 = FAISS(
    embedding_function=embeddings2,
    index=index2,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

uuids2 = [str(uuid4()) for _ in range(len(chunked_documents2))]
vector_store2.add_documents(documents=chunked_documents2, ids=uuids2)
vector_store2.save_local("faiss_index2")

## Generation Phase

In [9]:
vector_store = FAISS.load_local("faiss_index1", embeddings, allow_dangerous_deserialization=True)
vector_store2 = FAISS.load_local("faiss_index2", embeddings2, allow_dangerous_deserialization=True)

In [10]:
question = "How to add explainability to convolution and attention layers?"
results = vector_store.similarity_search(
    question,
    k=400
)

results2 = vector_store2.similarity_search(
    question,
    k=400
)


In [11]:
results = "\n".join(map(lambda i: i.page_content, results))
results2 = "\n".join(map(lambda i: i.page_content, results2))

In [12]:
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline


model_id = "gpt2"
hf_pipeline = pipeline("text-generation", model=model_id, max_new_tokens=1000000)

hf = HuggingFacePipeline(pipeline=hf_pipeline)


Device set to use cpu


## GPT‌4o Tests

In [14]:
from dotenv import load_dotenv


load_dotenv()

True

In [15]:
vector_store = FAISS.load_local("faiss_index1", embeddings, allow_dangerous_deserialization=True)
vector_store2 = FAISS.load_local("faiss_index2", embeddings2, allow_dangerous_deserialization=True)

In [16]:
question = "How to add explainability to convolution and attention layers?"
results = vector_store.similarity_search(
    question,
    k=40
)

results2 = vector_store2.similarity_search(
    question,
    k=40
)

In [17]:
results = "\n".join(map(lambda i: i.page_content, results2))
results2 = "\n".join(map(lambda i: i.page_content, results2))

In [18]:
from openai import OpenAI


client = OpenAI()
template = f"""
Answer the request using only the context below.
Context:
{results}

{results2}


Request: {question}
"""

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    store=True,
    messages=[
        {"role": "system", "content": "Combine only Context section provided and answer based on it only, summerizer it and dont use of your knowlegde in answer."},
        {"role": "user", "content": template}
    ]
)

In [19]:
print(completion.choices[0].message.content)

The provided context does not contain specific information about adding explainability to convolution and attention layers in machine learning models. It focuses on general principles of interpretability, the significance of contrastive explanations, the importance of social context in explanations, and methods for creating understandable explanations in machine learning. For insights on specific techniques or methods for explaining convolution and attention layers, further specialized sources would be necessary.


## Generation phase of new architecture

<img src='./preface.jpg'>

In [20]:
from dotenv import load_dotenv


load_dotenv()

True

In [21]:
vector_store = FAISS.load_local("faiss_index1", embeddings, allow_dangerous_deserialization=True)
vector_store2 = FAISS.load_local("faiss_index2", embeddings2, allow_dangerous_deserialization=True)

In [23]:
question = "How to add explainability to convolution and attention layers?"
results = vector_store.similarity_search(
    question,
    k=40
)

results2 = vector_store2.similarity_search(
    question,
    k=40
)

In [26]:
from openai import OpenAI


def query(question, results, client):
    template = f"""
    Answer the request using only the context below.
    Context:
    
    {"              ".join(results)}


    Request: {question}"""

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        store=True,
        messages=[
            {"role": "system", "content": "Combine only Context section provided and answer based on it only, summerizer it and dont use of your knowlegde in answer."},
            {"role": "user", "content": template}
        ]
    )
    return completion.choices[0].message.content

In [27]:
import time

client = OpenAI()


def group_elements(input_list, group_size):
    return ["\n".join(map(lambda l: l.page_content, input_list[i:i + group_size])) for i in range(0, len(input_list), group_size)]


input1 = group_elements(results, 10)
input2 = group_elements(results2, 10)
output = []
for res in zip(input1, input2):
    output.append(query(question, res, client))
    time.sleep(30)

print(output)

["Explainability can be added to convolution and attention layers through the use of attention mechanisms. These mechanisms help to clarify what aspects of the input data are influencing the model's predictions, which is crucial for understanding model behavior and improving its interpretability.\n\nIn the context of convolutional neural networks (CNNs), attention mechanisms can assist in revealing which parts of an image contribute significantly to a particular classification. For example, if a CNN misclassifies an image, examining the attention map can highlight which features attracted the model's focus, allowing for better debugging and retraining with more relevant data.\n\nSimilarly, in attention-based architectures like Transformers, explainability can be achieved as attention models prioritize different parts of the input at each time step in sequence data processing. By visualizing these attention weights, stakeholders can understand the model's reasoning behind specific outpu

In [28]:
import time

client = OpenAI()


def group_elements(input_list, group_size):
    return ["\n".join(map(lambda l: l.page_content, input_list[i:i + group_size])) for i in range(0, len(input_list), group_size)]


input1 = group_elements(results, 10)
input2 = group_elements(results2, 10)
input1.extend(input2)
output = []
for i in range(0, len(input1), 2):
    output.append(query(question, input1[i: i + 2], client))
    time.sleep(10)

print(output)

["Explainability in machine learning, particularly with convolutional and attention layers, can be enhanced by analyzing how these layers contribute to the model's predictions. For convolutional layers, you can check the feature maps produced during processing to understand what patterns the model is focusing on, such as specific shapes or textures in images. In contrast, attention mechanisms allow for a more explicit analysis of the model's focus. When the model generates an output, you can inspect the attention weights to see which parts of the input it considered most relevant. This approach helps clarify why the model made a particular decision, which can be crucial for debugging or meeting legal requirements in applications like loan approvals. This means both convolutional and attention layers can be utilized to gain insights into model behavior, facilitating improvements and ensuring compliance.", 'The context does not provide specific methods or techniques for adding explainabi

In [29]:
query(question, output, client)

"To add explainability to convolution and attention layers, one can analyze how these layers contribute to the model's predictions. For convolutional layers, examining the feature maps can reveal the patterns the model is focusing on, such as shapes or textures in images. For attention mechanisms, inspecting the attention weights can identify which parts of the input are deemed most relevant during output generation. Utilizing interpretability methods like local interpretable model-agnostic explanations (LIME) or model-agnostic methods like local surrogate models and Shapley values may also help generate concise explanations for predictions. It is essential to consider the audience's understanding and the social context of the explanations to ensure they are clear and effective."

In [30]:
from pprint import pprint
pprint("To enhance explainability in convolutional and attention layers, attention mechanisms can be applied. These mechanisms allow for an understanding of model outputs by identifying which parts of the input (such as images or text) the model concentrated on during predictions. For instance, if an image is misclassified, attention mechanisms can help analyze the model's focus, clarifying its reasoning.\n\nIn practice, visual attention can be incorporated, where a convolutional neural network processes an image and delivers feature maps to a decoder. The decoder, using an attention mechanism, generates outputs (like captions) sequentially while emphasizing relevant aspects of the input image.\n\nAdditionally, the Transformer architecture exemplifies the effectiveness of solely using attention mechanisms for understanding and explaining model predictions. By including attention layers, models become more interpretable, offering insights into their decision-making processes.")

('To enhance explainability in convolutional and attention layers, attention '
 'mechanisms can be applied. These mechanisms allow for an understanding of '
 'model outputs by identifying which parts of the input (such as images or '
 'text) the model concentrated on during predictions. For instance, if an '
 "image is misclassified, attention mechanisms can help analyze the model's "
 'focus, clarifying its reasoning.\n'
 '\n'
 'In practice, visual attention can be incorporated, where a convolutional '
 'neural network processes an image and delivers feature maps to a decoder. '
 'The decoder, using an attention mechanism, generates outputs (like captions) '
 'sequentially while emphasizing relevant aspects of the input image.\n'
 '\n'
 'Additionally, the Transformer architecture exemplifies the effectiveness of '
 'solely using attention mechanisms for understanding and explaining model '
 'predictions. By including attention layers, models become more '
 'interpretable, offering ins